# Multiple API Versions

**Purpose:** Determine to what extent Crossplane allows us to operate on multiple versions of an XRD in parallel.


## Findings
1. Only one version in an XRD may be `referenceable`.
2. After introducing a new version, we can continue to create resources using the old version, if the old version is `referenceable`.
3. After introducing a new version, if the new version is not `referenceable`, it seems impossible to successfully execute the composition for a new resource created on the new version.
4. After introducing a new version, if the new version is `referenceable`, we can create resources with the new version.

## Setup

Start with a clean cluster per `00 Setup` notebook.

Define an XRD with a single property `foo`:

In [59]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: CompositeResourceDefinition
metadata:
  name: xsimples.example.guidewire.net
spec:
  group: example.guidewire.net
  names:
    kind: xSimple
    plural: xsimples
  claimNames:
    kind: Simple
    plural: simples
  versions:
  - name: v1alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
EOF

compositeresourcedefinition.apiextensions.crossplane.io/xsimples.example.guidewire.net created


Let's also define an implementation that creates a `ConfigMap` with the value of `foo` put in it:

In [60]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition-v1alpha1
spec:
  mode: Pipeline
  compositeTypeRef:
    apiVersion: example.guidewire.net/v1alpha1
    kind: xSimple
  pipeline:
  - step: patch-and-transform
    functionRef:
      name: function-patch-and-transform
    input:
      apiVersion: pt.fn.crossplane.io/v1beta1
      kind: Resources
      resources:
      - name: themap
        base:
          apiVersion: kubernetes.crossplane.io/v1alpha2
          kind: Object
          spec:
            forProvider:
              manifest:
                apiVersion: v1
                kind: ConfigMap
                metadata:
                  namespace: default
                data:
                  constant: first version
                  foo: replace me
        patches:
          - type: FromCompositeFieldPath
            fromFieldPath: spec.foo
            toFieldPath: spec.forProvider.manifest.data.foo
EOF

composition.apiextensions.crossplane.io/simple-composition-v1alpha1 created


Now we can create a `Claim` and see that the `ConfigMap` gets created:

In [61]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test
  namespace: default
spec:
  foo: "Hello, World!"
  compositionRef:
    name: simple-composition-v1alpha1
EOF

simple.example.guidewire.net/test created


In [62]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        3s
-----------------
Name:         test-jk966-ccslc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


## Experiment: Introduce a New Version on the XRD

We define a new version on the XRD and a new `Composition` to implement it.

**Hypothesis:** The existing claim `test` continues to function. New claims with the new version can be created and function correctly. New claims with the old version can also be created, so that consumers can take time to transition to the new version without losing functionality.

Start by defining a new version, `v2alpha1`, with a new *required* attribute `bar`:

In [63]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: CompositeResourceDefinition
metadata:
  name: xsimples.example.guidewire.net
spec:
  group: example.guidewire.net
  names:
    kind: xSimple
    plural: xsimples
  claimNames:
    kind: Simple
    plural: simples
  versions:
  - name: v1alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
  - name: v2alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
              bar:
                type: string
            required:
              - bar
EOF

The CompositeResourceDefinition "xsimples.example.guidewire.net" is invalid: 
* <generated_CRD_"xsimples.example.guidewire.net">.spec.versions: Invalid value: []apiextensions.CustomResourceDefinitionVersion{apiextensions.CustomResourceDefinitionVersion{Name:"v1alpha1", Served:true, Storage:true, Deprecated:false, DeprecationWarning:(*string)(nil), Schema:(*apiextensions.CustomResourceValidation)(0x40096e29b0), Subresources:(*apiextensions.CustomResourceSubresources)(nil), AdditionalPrinterColumns:[]apiextensions.CustomResourceColumnDefinition(nil), SelectableFields:[]apiextensions.SelectableField(nil)}, apiextensions.CustomResourceDefinitionVersion{Name:"v2alpha1", Served:true, Storage:true, Deprecated:false, DeprecationWarning:(*string)(nil), Schema:(*apiextensions.CustomResourceValidation)(0x40096e29b8), Subresources:(*apiextensions.CustomResourceSubresources)(nil), AdditionalPrinterColumns:[]apiextensions.CustomResourceColumnDefinition(nil), SelectableFields:[]apiextensions.Selectab

CalledProcessError: Command 'b'cat << EOF | kubectl apply -f -\napiVersion: apiextensions.crossplane.io/v1\nkind: CompositeResourceDefinition\nmetadata:\n  name: xsimples.example.guidewire.net\nspec:\n  group: example.guidewire.net\n  names:\n    kind: xSimple\n    plural: xsimples\n  claimNames:\n    kind: Simple\n    plural: simples\n  versions:\n  - name: v1alpha1\n    served: true\n    referenceable: true\n    schema:\n      openAPIV3Schema:\n        type: object\n        properties:\n          spec:\n            type: object\n            properties:\n              foo:\n                type: string\n  - name: v2alpha1\n    served: true\n    referenceable: true\n    schema:\n      openAPIV3Schema:\n        type: object\n        properties:\n          spec:\n            type: object\n            properties:\n              foo:\n                type: string\n              bar:\n                type: string\n            required:\n              - bar\nEOF\n'' returned non-zero exit st

It did not work; the problem is that only one version may have `referenceable` as `true`. We correct this error so the XRD can be applied successfully:

In [64]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: CompositeResourceDefinition
metadata:
  name: xsimples.example.guidewire.net
spec:
  group: example.guidewire.net
  names:
    kind: xSimple
    plural: xsimples
  claimNames:
    kind: Simple
    plural: simples
  versions:
  - name: v1alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
  - name: v2alpha1
    served: true
    referenceable: false
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
              bar:
                type: string
            required:
              - bar
EOF

compositeresourcedefinition.apiextensions.crossplane.io/xsimples.example.guidewire.net configured


We introduce a new `Composition` that works for `v2alpha1` and handles the new `bar` property:

In [65]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition-v2alpha1
spec:
  mode: Pipeline
  compositeTypeRef:
    apiVersion: example.guidewire.net/v2alpha1
    kind: xSimple
  pipeline:
  - step: patch-and-transform
    functionRef:
      name: function-patch-and-transform
    input:
      apiVersion: pt.fn.crossplane.io/v1beta1
      kind: Resources
      resources:
      - name: themap
        base:
          apiVersion: kubernetes.crossplane.io/v1alpha2
          kind: Object
          spec:
            forProvider:
              manifest:
                apiVersion: v1
                kind: ConfigMap
                metadata:
                  namespace: default
                data:
                  constant: first version
                  foo: replace me
                  bar: gets replaced too
        patches:
          - type: FromCompositeFieldPath
            fromFieldPath: spec.foo
            toFieldPath: spec.forProvider.manifest.data.foo
          - type: FromCompositeFieldPath
            fromFieldPath: spec.bar
            toFieldPath: spec.forProvider.manifest.data.bar

composition.apiextensions.crossplane.io/simple-composition-v2alpha1 created


Confirm the first part of our hypothesis: the `test` claim created on `v1alpha1` is still present and working:

In [66]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl get claim test -o yaml | yq .apiVersion
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        29s
-----------------
example.guidewire.net/v2alpha1
-----------------
Name:         test-jk966-ccslc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


The `test` claim is still present, however: note it is served as version `v2alpha1`!

Confirm we can still create new resources with `v1alpha1`:

In [67]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test-v1
  namespace: default
spec:
  foo: "Hello, World!"
  compositionRef:
    name: simple-composition-v1alpha1
EOF

simple.example.guidewire.net/test-v1 created


In [69]:
%%bash
kubectl get claim
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME      SYNCED   READY   CONNECTION-SECRET   AGE
test      True     True                        2m31s
test-v1   True     True                        76s
-----------------
Name:         test-jk966-ccslc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Name:         test-v1-9kznd-9dqmc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Confirm the second part of our hypothesis: new claims can be created with version `v2alpha1`:

In [70]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v2alpha1
kind: Simple
metadata:
  name: test-v2
  namespace: default
spec:
  foo: "Hello, World!"
  bar: "Goodbye, Cruel World!"
  compositionRef:
    name: simple-composition-v2alpha1
EOF

simple.example.guidewire.net/test-v2 created


In [71]:
%%bash
kubectl get claim
kubectl describe `kubectl get configmap -o=name | grep test`

NAME      SYNCED   READY   CONNECTION-SECRET   AGE
test      True     True                        3m47s
test-v1   True     True                        2m32s
test-v2   True     False                       4s
Name:         test-jk966-ccslc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
foo:
----
Hello, World!
constant:
----
first version

BinaryData
====

Events:  <none>


Name:         test-v1-9kznd-9dqmc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


The new claim on `v2alpha1` is not ready, check the status of the underlying composite resource:

In [72]:
%%bash
kubectl describe composite `kubectl get claim test-v2 -o yaml | yq .spec.resourceRef.name`

Name:         test-v2-bkmq9
Namespace:    
Labels:       crossplane.io/claim-name=test-v2
              crossplane.io/claim-namespace=default
              crossplane.io/composite=test-v2-bkmq9
Annotations:  <none>
API Version:  example.guidewire.net/v2alpha1
Kind:         xSimple
Metadata:
  Creation Timestamp:  2024-12-20T04:43:13Z
  Finalizers:
    composite.apiextensions.crossplane.io
  Generate Name:     test-v2-
  Generation:        2
  Resource Version:  69881
  UID:               49e2b452-0128-488a-9e38-39412b123207
Spec:
  Claim Ref:
    API Version:  example.guidewire.net/v1alpha1
    Kind:         Simple
    Name:         test-v2
    Namespace:    default
  Composition Ref:
    Name:  simple-composition-v2alpha1
  Composition Revision Ref:
    Name:                     simple-composition-v2alpha1-83cb458
  Composition Update Policy:  Automatic
  Foo:                        Hello, World!
Status:
  Conditions:
    Last Transition Time:  2024-12-20T04:43:13Z
    Message:       

The error is:

`cannot configure composite resource: referenced composition is not compatible with this composite resource`

Hypothesis: the issue is that the `v2alpha1` version is not `referenceable`. Let's test that:

In [73]:
%%bash
kubectl delete claim test-v2

simple.example.guidewire.net "test-v2" deleted


In [74]:
%%bash
kubectl patch xrd xsimples.example.guidewire.net --type='json' -p='[{"op": "replace", "path": "/spec/versions/0/referenceable", "value":false},{"op": "replace", "path": "/spec/versions/1/referenceable", "value":true}]'

compositeresourcedefinition.apiextensions.crossplane.io/xsimples.example.guidewire.net patched


In [75]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v2alpha1
kind: Simple
metadata:
  name: test-v2
  namespace: default
spec:
  foo: "Hello, World!"
  bar: "Goodbye, Cruel World!"
  compositionRef:
    name: simple-composition-v2alpha1
EOF

simple.example.guidewire.net/test-v2 created


In [76]:
%%bash
kubectl get claim
kubectl describe `kubectl get configmap -o=name | grep test`

NAME      SYNCED   READY   CONNECTION-SECRET   AGE
test      False    True                        4m17s
test-v1   False    True                        3m2s
test-v2   True     True                        4s
Name:         test-jk966-ccslc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Name:         test-v1-9kznd-9dqmc
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Name:         test-v2-q4zj7-chplk
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
bar:
----
Goodbye, Cruel World!
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Unfortunately, we can now predict that creating a `v1alpha1` resource no longer works:

In [77]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test3
  namespace: default
spec:
  foo: "Hello, World!"
  compositionRef:
    name: simple-composition-v1alpha1
EOF

simple.example.guidewire.net/test3 created


In [78]:
%%bash
kubectl get claim
echo "-----------------"
kubectl describe claim test3

NAME      SYNCED   READY   CONNECTION-SECRET   AGE
test      False    True                        4m32s
test-v1   False    True                        3m17s
test-v2   True     True                        19s
test3     False                                3s
-----------------
Name:         test3
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  example.guidewire.net/v2alpha1
Kind:         Simple
Metadata:
  Creation Timestamp:  2024-12-20T04:43:59Z
  Finalizers:
    finalizer.apiextensions.crossplane.io
  Generation:        1
  Resource Version:  70072
  UID:               5acfb1ae-6254-4b07-9e8b-17808088fb4b
Spec:
  Composite Delete Policy:  Background
  Composition Ref:
    Name:  simple-composition-v1alpha1
  Foo:     Hello, World!
Status:
  Conditions:
    Last Transition Time:  2024-12-20T04:43:59Z
    Message:               cannot bind and sync claim with composite resource: cannot update claim: Simple.example.guidewire.net "test3" is invalid: spec.bar:

It didn't work but the failure was unexpected:

`cannot bind and sync claim with composite resource: cannot update claim: Simple.example.guidewire.net "test3" is invalid: spec.bar: Required value`

It seems we can no longer create resources with `v1alpha1`; it's almost as if they're getting converted to `v2alpha1` in the background.

This is likely due to the CRD `storage` property which is derived from `referenceable` on the XRD:

In [79]:
%%bash
kubectl get crd xsimples.example.guidewire.net  -o yaml | yq '.spec.versions[] | {.name: .storage}'

v1alpha1: false
v2alpha1: true


TODO: What if `v2alpha1` had `bar` as optional; would we have gotten farther?

TODO: We should next look at conversions in Kubernetes...